# Template Empresarial: Autoencoder para Detección de Anomalías

**Objetivo**: Este template proporciona una base completa para desarrollar modelos de autoencoder con telemetría y observabilidad empresarial usando Splunk DSDL.

**Versión**: 2.15  
**Última actualización**: Basado en mejores prácticas de desarrollo E2E

## ¿Qué es este Template?

Este notebook contiene un workflow completo de ejemplo para desarrollar modelos de autoencoder con:

- ✅ **Telemetría automática**: Métricas de entrenamiento e inferencia enviadas a Splunk
- ✅ **Observabilidad**: Monitoreo de rendimiento del modelo en producción
- ✅ **Preprocesamiento**: Normalización automática de datos
- ✅ **Detección de anomalías**: Cálculo automático de scores de anomalía
- ✅ **Helpers empresariales**: Uso de módulos estándar para tareas comunes

**Ejemplo**: Autoencoder para detección de anomalías usando Keras y TensorFlow.

## ⚠️ Importante: Exportación Automática

Al guardar este notebook, DSDL **automáticamente** exporta las funciones requeridas a un módulo Python que luego se invoca desde Splunk con comandos SPL como:

```spl
| fit MLTKContainer algo=mi_modelo ... into app:mi_modelo
| apply mi_modelo
| summary mi_modelo
```

**Funciones requeridas**: `init`, `fit`, `apply`, `summary`, `save`, `load`

**Para más información**: Consulta la Guía Completa Data Scientist E2E.

## 📦 Paso 0: Imports y Configuración

En este paso se definen todos los imports necesarios y la configuración del modelo.

**Importante**: 
- Los helpers empresariales se importan desde `/dltk/notebooks_custom/helpers`
- La configuración del modelo sigue el naming estándar: `{app_name}_{model_type}_{use_case}_{version}`

In [ ]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands

import json
import os
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Importar helpers empresariales
import sys
sys.path.append('/dltk/notebooks_custom/helpers')

try:
    from telemetry_helper import log_metrics, log_training_step, log_error
    print("✅ telemetry_helper importado")
except ImportError:
    print("⚠️  telemetry_helper no disponible (telemetría deshabilitada)")

try:
    from metrics_calculator import calculate_all_metrics
    print("✅ metrics_calculator importado")
except ImportError:
    print("⚠️  metrics_calculator no disponible")

try:
    from preprocessor import standard_preprocessing, apply_preprocessing
    print("✅ preprocessor importado")
except ImportError:
    print("⚠️  preprocessor no disponible")

# Global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

# Configuración del modelo (usando naming estándar)
# ⚠️ IMPORTANTE: Actualiza estos valores según tu modelo
APP_NAME = "app1"
MODEL_TYPE = "autoencoder"
USE_CASE = "demo_anomalias"  # Cambiar según tu caso de uso
VERSION = "v1"  # Incrementar en cada versión
MODEL_NAME = f"{APP_NAME}_{MODEL_TYPE}_{USE_CASE}_{VERSION}"

print(f"\n📦 Modelo configurado: {MODEL_NAME}")
print(f"✅ Imports completados")

In [ ]:
# THIS CELL IS NOT EXPORTED - Verificar versiones y configuración
print("=" * 60)
print("VERIFICACIÓN DE VERSIONES Y CONFIGURACIÓN")
print("=" * 60)
print(f"\n📦 Versiones de librerías:")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")
print(f"   - TensorFlow: {tf.__version__}")
print(f"   - Keras: {keras.__version__}")

print(f"\n📊 Configuración del modelo:")
print(f"   - Nombre: {MODEL_NAME}")
print(f"   - App: {APP_NAME}")
print(f"   - Tipo: {MODEL_TYPE}")
print(f"   - Caso de uso: {USE_CASE}")
print(f"   - Versión: {VERSION}")

print(f"\n📁 Directorio de modelos: {MODEL_DIRECTORY}")
print(f"   - Existe: {os.path.exists(MODEL_DIRECTORY)}")
if not os.path.exists(MODEL_DIRECTORY):
    print("   ⚠️  Creando directorio...")
    os.makedirs(MODEL_DIRECTORY, exist_ok=True)

## 📥 Paso 1: Obtener Datos de Splunk (Opcional)

Este paso es opcional y solo se usa durante el desarrollo local. En producción, DSDL pasa los datos directamente a las funciones.

**Para desarrollo local**: Puedes usar `SplunkSearch` para obtener datos directamente desde Splunk.

### Ejemplo de SPL para desarrollo local:

```spl
index=demo_anomalias_data
| head 1000
| fit MLTKContainer algo=mi_modelo mode=stage epochs=50 batch_size=32 encoding_dim=10 from feature_* into app:mi_modelo
```

**Explicación**:
- `index=demo_anomalias_data`: Índice de Splunk con tus datos
- `head 1000`: Limitar a 1000 muestras para desarrollo
- `algo=mi_modelo`: Nombre del notebook (sin .ipynb)
- `mode=stage`: Modo de desarrollo (carga datos al notebook)
- `from feature_*`: Selecciona todas las columnas que empiezan con `feature_`
- `into app:mi_modelo`: Nombre del modelo guardado

### Ejemplo: Obtener Datos Directamente desde Splunk en JupyterLab

Puedes obtener datos directamente desde Splunk usando `SplunkSearch` sin necesidad de usar `stage()`.

**⚠️ IMPORTANTE**: El proceso es **interactivo** y requiere **dos celdas separadas**:

1. **Primera celda**: Crear y ejecutar la búsqueda en Splunk (el usuario hace clic en "Search")
2. **Segunda celda**: Una vez que la búsqueda completa, obtener el DataFrame con los resultados

**Ventajas de usar SplunkSearch**:
- ✅ No necesitas ejecutar `mode=stage` desde Splunk
- ✅ Obtienes datos directamente en JupyterLab
- ✅ Puedes ver los resultados en la UI de Splunk antes de convertirlos a DataFrame
- ✅ Puedes ajustar la búsqueda SPL fácilmente
- ✅ Útil para desarrollo y pruebas rápidas

**Nota**: En producción, DSDL pasa los datos directamente a `init()`, `fit()`, y `apply()`, así que esta función `stage()` solo es útil para desarrollo local cuando usas `mode=stage`.

In [ ]:
# THIS CELL IS NOT EXPORTED - Función stage() para desarrollo local
# ⚠️ NOTA: Esta función NO se exporta al .py porque solo se usa para desarrollo local.
# En producción, DSDL pasa los datos directamente a init(), fit(), apply().

def stage(name):
    """
    Cargar datos desde archivos CSV/JSON para desarrollo local.
    
    Esta función solo se usa durante el desarrollo en JupyterLab cuando ejecutas
    un comando SPL con 'mode=stage'. En producción, DSDL pasa los datos directamente.
    
    Args:
        name: Nombre del modelo (del parámetro 'into app:nombre' en SPL)
    
    Returns:
        tuple: (DataFrame, parámetros) - Datos y parámetros del modelo
    """
    try:
        with open("data/" + name + ".csv", 'r') as f:
            df = pd.read_csv(f)
        print(f"✅ Datos cargados: {df.shape}")
        
        with open("data/" + name + ".json", 'r') as f:
            param = json.load(f)
        print(f"✅ Parámetros cargados")
        
        return df, param
    except FileNotFoundError as e:
        print(f"⚠️  Archivo no encontrado: {e}")
        print("   Asegúrate de haber ejecutado el comando SPL con 'mode=stage' primero")
        print("   O usa SplunkSearch directamente (ver ejemplo en celda anterior)")
        raise

In [ ]:
# THIS CELL IS NOT EXPORTED - Paso 1: Ejecutar búsqueda en Splunk
# ⚠️ IMPORTANTE: Esta celda crea la búsqueda y la ejecuta en Splunk.
# El usuario debe hacer clic en el botón "Search" en la UI de Splunk que aparece.
# Una vez que la búsqueda completa, ejecuta la siguiente celda para obtener el DataFrame.

from dsdlsupport import SplunkSearch

# Definir la búsqueda SPL
# ⚠️ NOTA: Ajusta esta búsqueda según tus datos y necesidades
search_query = 'index=demo_anomalias_data | head 1000 | table feature_*'

print("🔍 Ejecutando búsqueda en Splunk...")
print(f"📋 Query: {search_query}")
print("\n⚠️  IMPORTANTE:")
print("   1. Se abrirá una ventana/UI de Splunk con los resultados")
print("   2. Haz clic en el botón 'Search' para ejecutar la búsqueda")
print("   3. Espera a que la búsqueda complete")
print("   4. Una vez completada, ejecuta la siguiente celda para obtener el DataFrame\n")

# Crear y ejecutar la búsqueda (interactivo)
search = SplunkSearch.SplunkSearch(search=search_query)

print("✅ Búsqueda creada. Haz clic en 'Search' en la UI de Splunk.")
print("📝 Una vez que la búsqueda complete, ejecuta la siguiente celda.")

In [ ]:
# THIS CELL IS NOT EXPORTED - Paso 2: Obtener DataFrame de la búsqueda completada
# ⚠️ IMPORTANTE: Ejecuta esta celda SOLO después de que la búsqueda anterior haya completado
# (después de hacer clic en "Search" y ver los resultados en la UI de Splunk)

try:
    # Verificar que la búsqueda existe
    if 'search' not in globals():
        print("❌ Error: No se encontró el objeto 'search'")
        print("   Ejecuta primero la celda anterior para crear la búsqueda")
    else:
        print("📥 Obteniendo DataFrame de los resultados de Splunk...")
        
        # Convertir resultados de Splunk a DataFrame
        df = search.as_df()
        
        print(f"✅ Datos obtenidos: {df.shape}")
        print(f"📊 Primeras filas:")
        print(df.head())
        print(f"\n📋 Columnas disponibles: {list(df.columns)}")
        
        # Crear parámetros manualmente para desarrollo local
        # ⚠️ NOTA: Ajusta estos parámetros según tus necesidades
        param = {
            'feature_variables': [col for col in df.columns if col.startswith('feature_')],
            'options': {
                'params': {
                    'epochs': '50',
                    'batch_size': '32',
                    'encoding_dim': '10',
                    'validation_split': '0.2'
                }
            }
        }
        
        print(f"\n⚙️  Parámetros creados:")
        print(json.dumps(param, indent=2))
        
        print("\n✅ DataFrame 'df' y parámetros 'param' listos para usar en init(), fit(), apply()")
        print("   Puedes continuar con las siguientes celdas del notebook")
        
except AttributeError as e:
    print(f"❌ Error: {e}")
    print("\n💡 Solución:")
    print("   1. Asegúrate de haber ejecutado la celda anterior")
    print("   2. Asegúrate de haber hecho clic en 'Search' en la UI de Splunk")
    print("   3. Espera a que la búsqueda complete antes de ejecutar esta celda")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


## 🔧 Paso 2: Inicializar el Modelo (`init`)

La función `init()` se llama automáticamente por DSDL antes de `fit()`. Su propósito es crear e inicializar la arquitectura del modelo.

**⚠️ IMPORTANTE**: Esta celda debe tener el metadata `"name": "mltkc_init"` para que DSDL la exporte correctamente.

In [ ]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently

def init(df, param):
    """
    Inicializar autoencoder para detección de anomalías.
    
    Esta función es llamada automáticamente por DSDL antes de fit().
    Crea la arquitectura del autoencoder y la compila.
    
    Args:
        df: DataFrame con datos de Splunk
        param: Diccionario con parámetros del modelo
            - feature_variables: Lista de columnas a usar como features
            - options.params.encoding_dim (opcional): Dimensión de la capa oculta (default: 10)
            - options.params.components (opcional): Alias para encoding_dim
            - options.params.activation (opcional): Función de activación (default: 'relu')
    
    Returns:
        model: Modelo Keras compilado listo para entrenar
    """
    print(f"🔧 Inicializando modelo: {MODEL_NAME}")
    
    # Obtener features del DataFrame
    if 'feature_variables' in param:
        feature_cols = param['feature_variables']
    else:
        # Si no hay feature_variables definidas, usar todas las numéricas
        feature_cols = [col for col in df.columns if df[col].dtype in ['float64', 'int64']]
        if not feature_cols:
            # Fallback: buscar columnas que empiecen con 'feature_'
            feature_cols = [col for col in df.columns if col.startswith('feature_')]
    
    X = df[feature_cols] if feature_cols else df.select_dtypes(include=[np.number])
    
    print(f"📊 Shape de los datos: {X.shape}")
    print(f"📋 Features seleccionadas: {len(X.columns)}")
    
    input_dim = X.shape[1]
    
    # Parámetros del modelo (con valores por defecto)
    encoding_dim = 10  # Dimensión de la capa oculta (bottleneck)
    if 'options' in param and 'params' in param['options']:
        if 'encoding_dim' in param['options']['params']:
            encoding_dim = int(param['options']['params']['encoding_dim'])
        elif 'components' in param['options']['params']:
            encoding_dim = int(param['options']['params']['components'])
    
    activation = 'relu'
    if 'options' in param and 'params' in param['options']:
        if 'activation' in param['options']['params']:
            activation = param['options']['params']['activation']
        elif 'activation_func' in param['options']['params']:
            activation = param['options']['params']['activation_func']
    
    print(f"⚙️  Parámetros del modelo:")
    print(f"   - Input dimension: {input_dim}")
    print(f"   - Encoding dimension: {encoding_dim}")
    print(f"   - Activation: {activation}")
    
    # Construir autoencoder
    # Encoder: reduce dimensiones
    encoder = keras.layers.Dense(
        encoding_dim, 
        activation=activation,
        input_shape=(input_dim,),
        name='encoder'
    )
    
    # Decoder: reconstruye dimensiones originales
    decoder = keras.layers.Dense(
        input_dim,
        activation=activation,
        name='decoder'
    )
    
    # Modelo completo
    model = keras.Sequential([
        encoder,
        decoder
    ], name='Autoencoder')
    
    # Compilar modelo
    model.compile(
        optimizer='adam',
        loss='mse',  # Mean Squared Error para autoencoder
        metrics=['mae']  # Mean Absolute Error como métrica adicional
    )
    
    print(f"✅ Modelo compilado exitosamente")
    print(f"📐 Arquitectura: {input_dim} → {encoding_dim} → {input_dim}")
    
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - Test init localmente
# ⚠️ NOTA: Solo funciona si tienes datos cargados (stage() o SplunkSearch)

try:
    # Crear datos dummy para prueba si no hay datos reales
    if 'df' not in globals() or 'param' not in globals():
        print("⚠️  No hay datos cargados. Creando datos dummy para prueba...")
        test_df = pd.DataFrame({
            'feature_0': np.random.randn(100),
            'feature_1': np.random.randn(100),
            'feature_2': np.random.randn(100),
            'feature_3': np.random.randn(100),
            'feature_4': np.random.randn(100)
        })
        test_param = {
            'feature_variables': ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4'],
            'options': {
                'params': {
                    'encoding_dim': 10
                }
            }
        }
        test_model = init(test_df, test_param)
        print("\n📊 Resumen del modelo de prueba:")
        test_model.summary()
    else:
        model = init(df, param)
        print("\n📊 Resumen del modelo:")
        model.summary()
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## 🏋️ Paso 3: Entrenar el Modelo (`fit`)

La función `fit()` se llama automáticamente por DSDL para entrenar el modelo. Incluye:

- ✅ Preprocesamiento automático (normalización)
- ✅ Telemetría por época (métricas enviadas a Splunk)
- ✅ Callback de TensorBoard para visualización
- ✅ Cálculo de métricas de reconstrucción
- ✅ Guardado del scaler para uso posterior

**⚠️ IMPORTANTE**: Esta celda debe tener el metadata `"name": "mltkc_stage_create_model_fit"` para que DSDL la exporte correctamente.

In [ ]:
# mltkc_stage_create_model_fit
# returns a fit info json object

def fit(model, df, param):
    """
    Entrenar autoencoder con telemetría automática.
    
    Esta función es llamada automáticamente por DSDL para entrenar el modelo.
    Incluye preprocesamiento, telemetría por época, y cálculo de métricas.
    
    Args:
        model: Modelo Keras inicializado (retornado por init())
        df: DataFrame con datos de entrenamiento
        param: Diccionario con parámetros de entrenamiento
            - feature_variables: Lista de columnas a usar como features
            - options.params.epochs (opcional): Número de épocas (default: 50)
            - options.params.batch_size (opcional): Tamaño de batch (default: 32)
            - options.params.validation_split (opcional): Fracción de validación (default: 0.2)
    
    Returns:
        dict: Información del entrenamiento
            - fit_history: Historial de entrenamiento de Keras
            - scaler: Scaler usado para normalización (CRÍTICO para apply())
            - model_loss: Loss final del modelo
            - model_mae: MAE final del modelo
            - mse: Mean Squared Error de reconstrucción
            - rmse: Root Mean Squared Error de reconstrucción
    """
    print(f"🚀 Iniciando entrenamiento del modelo: {MODEL_NAME}")
    
    returns = {}
    
    # Obtener features
    if 'feature_variables' in param:
        feature_cols = param['feature_variables']
    else:
        feature_cols = [col for col in df.columns if col.startswith('feature_')]
        if not feature_cols:
            feature_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    X = df[feature_cols] if feature_cols else df.select_dtypes(include=[np.number])
    
    print(f"📊 Datos de entrenamiento: {X.shape[0]} muestras, {X.shape[1]} features")
    
    # Preprocesamiento: Normalización (CRÍTICO para autoencoders)
    print("🔧 Aplicando preprocesamiento (normalización)...")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    
    # Guardar scaler en returns para uso posterior en apply()
    returns['scaler'] = scaler
    
    # Parámetros de entrenamiento
    epochs = 50
    batch_size = 32
    validation_split = 0.2
    
    if 'options' in param and 'params' in param['options']:
        if 'epochs' in param['options']['params']:
            epochs = int(param['options']['params']['epochs'])
        if 'batch_size' in param['options']['params']:
            batch_size = int(param['options']['params']['batch_size'])
        if 'validation_split' in param['options']['params']:
            validation_split = float(param['options']['params']['validation_split'])
    
    print(f"⚙️  Parámetros de entrenamiento:")
    print(f"   - Epochs: {epochs}")
    print(f"   - Batch size: {batch_size}")
    print(f"   - Validation split: {validation_split}")
    
    # Callback para TensorBoard (opcional, para visualización)
    log_dir = f"/srv/notebooks/logs/fit/{MODEL_NAME}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1
    )
    
    # Callback personalizado para logging de telemetría por época
    class TelemetryCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            """Enviar métricas de cada época a Splunk"""
            logs = logs or {}
            try:
                # ⚠️ CRÍTICO: Convertir valores NumPy a tipos nativos de Python para JSON serialization
                epoch_value = int(epoch + 1)  # Convertir a int nativo
                loss_value = float(logs.get('loss', 0)) if logs.get('loss') is not None else 0.0
                val_loss_value = float(logs.get('val_loss', 0)) if logs.get('val_loss') is not None else 0.0
                mae_value = float(logs.get('mae', 0)) if logs.get('mae') is not None else 0.0
                val_mae_value = float(logs.get('val_mae', 0)) if logs.get('val_mae') is not None else 0.0
                
                # Intentar usar log_training_step si está disponible
                try:
                    log_training_step(
                        model_name=MODEL_NAME,
                        epoch=epoch_value,
                        loss=loss_value,
                        val_loss=val_loss_value,
                        mae=mae_value,
                        val_mae=val_mae_value
                    )
                except NameError:
                    # Si log_training_step no está disponible, no hacer nada
                    pass
            except Exception as e:
                print(f"⚠️  Error enviando telemetría en época {epoch + 1}: {e}")
                import traceback
                print(f"   Traceback completo: {traceback.format_exc()}")
    
    telemetry_callback = TelemetryCallback()
    
    # Entrenar modelo
    print("\n🏋️  Iniciando entrenamiento...")
    history = model.fit(
        x=X_scaled_df,
        y=X_scaled_df,  # Autoencoder: input = output
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        verbose=1,
        callbacks=[tensorboard_callback, telemetry_callback]
    )
    
    returns['fit_history'] = history
    returns['model_epochs'] = epochs
    returns['model_batch_size'] = batch_size
    returns['scaler'] = scaler  # Guardar scaler para uso en apply
    
    # Evaluar modelo en datos completos
    print("\n📊 Evaluando modelo en datos completos...")
    test_results = model.evaluate(X_scaled_df, X_scaled_df, verbose=0)
    returns['model_loss'] = float(test_results[0])  # Convertir a float nativo
    returns['model_mae'] = float(test_results[1]) if len(test_results) > 1 else None
    
    print(f"✅ Entrenamiento completado")
    print(f"   - Loss final: {test_results[0]:.6f}")
    if len(test_results) > 1:
        print(f"   - MAE final: {test_results[1]:.6f}")
    
    # Calcular métricas de reconstrucción
    print("\n📈 Calculando métricas de reconstrucción...")
    X_pred = model.predict(X_scaled_df, verbose=0)
    
    # Calcular MSE y RMSE
    mse = mean_squared_error(X_scaled_df.values, X_pred)
    rmse = np.sqrt(mse)
    
    returns['mse'] = float(mse)  # Convertir a float nativo
    returns['rmse'] = float(rmse)  # Convertir a float nativo
    
    print(f"   - MSE: {mse:.6f}")
    print(f"   - RMSE: {rmse:.6f}")
    
    # Enviar métricas finales a Splunk (telemetría)
    try:
        # ⚠️ CRÍTICO: Convertir valores NumPy a tipos nativos de Python para JSON serialization
        mae_value = float(returns['model_mae']) if returns['model_mae'] is not None else None
        rmse_value = float(rmse) if rmse is not None else None
        mse_value = float(mse) if mse is not None else None
        loss_value = float(test_results[0]) if test_results[0] is not None else None
        
        # Intentar usar log_metrics si está disponible
        try:
            log_metrics(
                model_name=MODEL_NAME,
                r2_score=None,  # Autoencoder no tiene R² tradicional
                mae=mae_value,
                rmse=rmse_value,
                mse=mse_value,
                loss=loss_value,
                app_name=APP_NAME,
                model_version=VERSION,
                project=USE_CASE
            )
            print("✅ Métricas enviadas a Splunk")
        except NameError:
            # Si log_metrics no está disponible, no hacer nada
            pass
    except Exception as e:
        print(f"⚠️  Error enviando métricas a Splunk: {e}")
        import traceback
        print(f"   Traceback completo: {traceback.format_exc()}")
    
    return returns

In [ ]:
# THIS CELL IS NOT EXPORTED - Test fit localmente
# ⚠️ NOTA: Solo funciona si tienes modelo y datos cargados

try:
    if 'model' not in globals() or 'df' not in globals() or 'param' not in globals():
        print("⚠️  No hay modelo o datos cargados. Creando datos dummy para prueba...")
        test_df = pd.DataFrame({
            'feature_0': np.random.randn(500),
            'feature_1': np.random.randn(500),
            'feature_2': np.random.randn(500),
            'feature_3': np.random.randn(500),
            'feature_4': np.random.randn(500)
        })
        test_param = {
            'feature_variables': ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4'],
            'options': {
                'params': {
                    'epochs': '10',  # Pocas épocas para prueba rápida
                    'batch_size': '32',
                    'validation_split': '0.2'
                }
            }
        }
        test_model = init(test_df, test_param)
        print("\n⏳ Entrenando modelo de prueba (esto tomará unos minutos)...")
        fit_results = fit(test_model, test_df, test_param)
    else:
        print("\n⏳ Entrenando modelo (esto tomará varios minutos)...")
        fit_results = fit(model, df, param)
    
    print("\n✅ Test de fit completado exitosamente")
    print(f"   - Loss: {fit_results.get('model_loss', 'N/A')}")
    print(f"   - MSE: {fit_results.get('mse', 'N/A')}")
    print(f"   - RMSE: {fit_results.get('rmse', 'N/A')}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# THIS CELL IS NOT EXPORTED - Explorar modelo entrenado
# ⚠️ NOTA: Solo funciona después de entrenar el modelo

try:
    if 'model' in globals():
        print("📊 Información del modelo:")
        print(f"   - Input shape: {model.input_shape}")
        print(f"   - Output shape: {model.output_shape}")
        print(f"   - Total params: {model.count_params()}")
        print(f"\n📐 Arquitectura:")
        model.summary()
    else:
        print("⚠️  Modelo no disponible. Ejecuta primero init() y fit()")
except Exception as e:
    print(f"❌ Error: {e}")

## 🔮 Paso 4: Aplicar el Modelo (`apply`)

La función `apply()` se llama automáticamente por DSDL para hacer inferencia con datos nuevos. Incluye:

- ✅ Detección de anomalías basada en error de reconstrucción
- ✅ Cálculo de scores de anomalía normalizados
- ✅ Telemetría de inferencia (métricas enviadas a Splunk)
- ✅ Uso del scaler guardado durante fit()

**⚠️ IMPORTANTE**: Esta celda debe tener el metadata `"name": "mltkc_stage_create_model_apply"` para que DSDL la exporte correctamente.

In [ ]:
# mltkc_stage_create_model_apply

def apply(model, df, param):
    """
    Aplicar autoencoder para detección de anomalías.
    
    Esta función es llamada automáticamente por DSDL para hacer inferencia con datos nuevos.
    Calcula el error de reconstrucción y detecta anomalías basándose en un umbral.
    
    Args:
        model: Modelo Keras entrenado (retornado por fit())
        df: DataFrame con datos nuevos para inferencia
        param: Diccionario con parámetros (debe contener scaler de fit())
            - feature_variables: Lista de columnas a usar como features
            - scaler: Scaler usado durante fit() (CRÍTICO - debe venir de fit())
            - anomaly_threshold (opcional): Umbral de anomalía (default: percentil 95)
    
    Returns:
        DataFrame: DataFrame con reconstrucciones y scores de anomalía
            - reconstruction_error: Error de reconstrucción por muestra
            - anomaly_score: Score normalizado de anomalía
            - is_anomaly: 1 si es anomalía, 0 si no
            - reconstruction_*: Columnas con valores reconstruidos
            - original_*: Columnas con valores originales
    """
    print(f"🔮 Aplicando modelo: {MODEL_NAME}")
    
    # Obtener features (debe coincidir con las usadas en fit)
    if 'feature_variables' in param:
        feature_cols = param['feature_variables']
    else:
        feature_cols = [col for col in df.columns if col.startswith('feature_')]
        if not feature_cols:
            feature_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    X = df[feature_cols] if feature_cols else df.select_dtypes(include=[np.number])
    
    print(f"📊 Datos de inferencia: {X.shape[0]} muestras, {X.shape[1]} features")
    
    # Obtener scaler del entrenamiento (desde param o fit_results)
    scaler = None
    if 'scaler' in param:
        scaler = param['scaler']
    elif hasattr(model, 'scaler'):
        scaler = model.scaler
    
    # Aplicar normalización
    if scaler is not None:
        # Usar scaler del entrenamiento
        X_scaled = scaler.transform(X)
        print("✅ Usando scaler del entrenamiento")
    else:
        # Crear nuevo scaler si no está disponible (fallback)
        print("⚠️  Scaler no encontrado en param. Aplicando normalización nueva...")
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
    
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    
    # Predecir reconstrucciones
    print("🔄 Calculando reconstrucciones...")
    X_reconstructed = model.predict(X_scaled_df, verbose=0)
    X_reconstructed_df = pd.DataFrame(X_reconstructed, columns=X.columns, index=X.index)
    
    # Calcular error de reconstrucción (MSE por muestra)
    reconstruction_error = np.mean((X_scaled_df.values - X_reconstructed_df.values) ** 2, axis=1)
    
    # Calcular threshold para anomalías (percentil 95)
    # En producción, este threshold debería venir del conjunto de entrenamiento
    if 'anomaly_threshold' in param:
        anomaly_threshold = float(param['anomaly_threshold'])
    else:
        anomaly_threshold = float(np.percentile(reconstruction_error, 95))
    
    # Detectar anomalías
    is_anomaly = (reconstruction_error > anomaly_threshold).astype(int)
    anomaly_score = reconstruction_error / (anomaly_threshold + 1e-10)  # Normalizar score
    
    print(f"📊 Estadísticas de reconstrucción:")
    print(f"   - Error medio: {np.mean(reconstruction_error):.6f}")
    print(f"   - Error mediano: {np.median(reconstruction_error):.6f}")
    print(f"   - Threshold (percentil 95): {anomaly_threshold:.6f}")
    print(f"   - Anomalías detectadas: {is_anomaly.sum()} / {len(is_anomaly)} ({100*np.mean(is_anomaly):.2f}%)")
    
    # Construir DataFrame de resultados
    results = pd.DataFrame({
        'reconstruction_error': reconstruction_error,
        'anomaly_score': anomaly_score,
        'is_anomaly': is_anomaly
    }, index=X.index)
    
    # Agregar reconstrucciones como columnas (opcional)
    for i, col in enumerate(X.columns):
        results[f'reconstruction_{col}'] = X_reconstructed_df[col].values
        results[f'original_{col}'] = X[col].values
    
    print(f"✅ Inferencia completada. Shape de resultados: {results.shape}")
    
    # Enviar telemetría de inferencia a Splunk
    try:
        # ⚠️ CRÍTICO: Convertir valores NumPy a tipos nativos de Python para JSON serialization
        num_predictions = int(len(df))  # len() ya retorna int nativo
        num_anomalies = int(is_anomaly.sum())
        avg_reconstruction_error = float(np.mean(reconstruction_error))
        anomaly_threshold_native = float(anomaly_threshold)
        
        # Preparar datos de telemetría
        telemetry_data = {
            "model_name": MODEL_NAME,
            "num_predictions": num_predictions,
            "num_anomalies": num_anomalies,
            "avg_reconstruction_error": avg_reconstruction_error,
            "anomaly_threshold": anomaly_threshold_native,
            "app_name": APP_NAME,
            "model_version": VERSION,
            "project": USE_CASE
        }
        
        # Eliminar valores None
        telemetry_data = {k: v for k, v in telemetry_data.items() if v is not None}
        
        # Intentar usar log_prediction si está disponible, sino log_metrics
        try:
            from telemetry_helper import log_prediction
            log_prediction(**telemetry_data)
            print("✅ Telemetría de inferencia enviada a Splunk (usando log_prediction)")
        except (ImportError, NameError):
            try:
                log_metrics(**telemetry_data)
                print("✅ Telemetría de inferencia enviada a Splunk (usando log_metrics)")
            except NameError:
                # Si ninguna función está disponible, no hacer nada
                pass
    except Exception as e:
        print(f"⚠️  Error enviando telemetría de inferencia a Splunk: {e}")
        import traceback
        print(f"   Traceback completo: {traceback.format_exc()}")
    
    return results

In [ ]:
# THIS CELL IS NOT EXPORTED - Test apply localmente
# ⚠️ NOTA: Solo funciona si tienes modelo entrenado y datos cargados

try:
    if 'model' not in globals() or 'fit_results' not in globals():
        print("⚠️  No hay modelo entrenado. Ejecuta primero init() y fit()")
    else:
        # Crear datos nuevos para inferencia
        test_df_apply = pd.DataFrame({
            'feature_0': np.random.randn(100),
            'feature_1': np.random.randn(100),
            'feature_2': np.random.randn(100),
            'feature_3': np.random.randn(100),
            'feature_4': np.random.randn(100)
        })
        
        # Agregar scaler al param (simulando que viene de fit)
        test_param_apply = {
            'feature_variables': ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4'],
            'scaler': fit_results.get('scaler')  # Usar scaler del fit anterior
        }
        
        # Aplicar modelo
        results = apply(model, test_df_apply, test_param_apply)
        
        print("\n📊 Primeras 10 filas de resultados:")
        print(results.head(10))
        
        print("\n📈 Estadísticas de anomalías:")
        print(f"   - Total muestras: {len(results)}")
        print(f"   - Anomalías detectadas: {results['is_anomaly'].sum()}")
        print(f"   - Porcentaje: {100 * results['is_anomaly'].mean():.2f}%")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## 💾 Paso 5: Guardar el Modelo (`save`)

La función `save()` se llama **automáticamente** por DSDL después de `fit()`.

**⚠️ CRÍTICO**: Esta función es **REQUERIDA** por DSDL. Si no existe o no tiene el metadata correcto, verás el error:
```
AttributeError: module 'app.model.mi_modelo' has no attribute 'save'
```

**⚠️ IMPORTANTE**: Esta celda debe tener el metadata `"name": "mltkc_save"` para que DSDL la exporte correctamente.

In [ ]:
# mltkc_save
# Función REQUERIDA: DSDL llama a save(model, name) después de fit()

def save(model, name):
    """
    Guardar modelo Keras en disco.
    
    IMPORTANTE: Esta función es llamada automáticamente por DSDL después de fit().
    DSDL usa el nombre pasado desde 'into app:nombre' en SPL.
    
    Args:
        model: Modelo Keras entrenado (retornado por fit())
        name: Nombre del modelo (pasado por DSDL desde "into app:model_name")
    
    Returns:
        model: Retorna el modelo (requerido por DSDL)
    """
    import os
    
    # Asegurar que el directorio existe
    os.makedirs(MODEL_DIRECTORY, exist_ok=True)
    
    # Guardar modelo Keras
    filepath = MODEL_DIRECTORY + name + ".keras"
    model.save(filepath)
    
    print(f"✅ Modelo guardado en: {filepath}")
    print(f"📊 Tamaño del archivo: {os.path.getsize(filepath) / (1024*1024):.2f} MB")
    
    # NOTA: Si tienes un scaler u otros objetos, guárdalos también
    # Ejemplo: si el scaler está en el modelo o en globals
    # from sklearn.externals import joblib  # o import joblib
    # if hasattr(model, 'scaler'):
    #     joblib.dump(model.scaler, MODEL_DIRECTORY + name + "_scaler.pkl")
    
    # DSDL espera que retornes el modelo
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - Test save localmente (opcional)
# ⚠️ NOTA: En producción, DSDL llama a save() automáticamente después de fit()

try:
    if 'model' not in globals():
        print("⚠️  No hay modelo disponible. Ejecuta primero init() y fit()")
    else:
        # Guardar modelo de prueba usando la firma correcta
        saved_model = save(model, name="test_autoencoder")
        print(f"✅ Modelo guardado exitosamente")
        
        # Verificar que el archivo existe
        filepath = MODEL_DIRECTORY + "test_autoencoder.keras"
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024 * 1024)
            print(f"📊 Tamaño del archivo: {file_size:.2f} MB")
            print(f"✅ Archivo creado correctamente: {filepath}")
        else:
            print(f"⚠️  Archivo no encontrado: {filepath}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## 📥 Paso 6: Cargar el Modelo (`load`) - Opcional

La función `load()` es **opcional** y solo se usa para desarrollo local. DSDL **NO** la llama automáticamente.

**Uso**: Útil para cargar un modelo guardado durante desarrollo o pruebas.

**⚠️ NOTA**: Si quieres que DSDL exporte esta función, agrega el metadata `"name": "mltkc_load"` a la celda.

In [ ]:
# mltkc_load
# Función opcional para cargar modelo guardado durante desarrollo
# DSDL NO llama a esta función automáticamente

def load(name):
    """
    Cargar modelo Keras desde disco.
    
    Útil para desarrollo local o pruebas.
    DSDL NO usa esta función automáticamente.
    
    Args:
        name: Nombre del archivo (sin extensión)
    
    Returns:
        Model: Modelo Keras cargado
    """
    import os
    
    try:
        model_dir = MODEL_DIRECTORY
    except NameError:
        model_dir = "/srv/app/model/data/"
    
    filepath = model_dir + name + ".keras"
    
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"❌ Archivo no encontrado: {filepath}")
    
    print(f"📥 Cargando modelo desde: {filepath}")
    model = keras.models.load_model(filepath)
    
    print(f"✅ Modelo cargado exitosamente")
    print(f"📊 Arquitectura: {model.input_shape} → {model.output_shape}")
    
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - Test load localmente (opcional)
# ⚠️ NOTA: Requiere que hayas guardado un modelo primero

try:
    test_filepath = MODEL_DIRECTORY + "test_autoencoder.keras"
    if not os.path.exists(test_filepath):
        print(f"⚠️  Archivo no encontrado: {test_filepath}")
        print("   Necesitas ejecutar primero el test de save() (Paso 5)")
        print("   O asegúrate de que test_autoencoder.keras existe")
    else:
        loaded_model = load("test_autoencoder")
        print("✅ Modelo cargado exitosamente")
        
        # Verificar que son equivalentes (solo si model existe)
        if 'model' in globals():
            print("\n🔍 Verificando que el modelo cargado funciona...")
            test_input = np.random.randn(1, 5)  # 5 features
            output_original = model.predict(test_input, verbose=0)
            output_loaded = loaded_model.predict(test_input, verbose=0)
            
            if np.allclose(output_original, output_loaded):
                print("✅ Los modelos producen resultados idénticos")
            else:
                print("⚠️  Los modelos producen resultados diferentes")
        else:
            print("⚠️  model no está definido, no se puede verificar equivalencia")
            print("   Pero el modelo se cargó correctamente ✅")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## 📊 Paso 7: Resumen del Modelo (`summary`)

La función `summary()` se llama automáticamente por DSDL para obtener metadatos del modelo.

**⚠️ IMPORTANTE**: Esta celda debe tener el metadata `"name": "mltkc_summary"` para que DSDL la exporte correctamente.

**⚠️ CRÍTICO**: Todos los valores NumPy deben convertirse a tipos nativos de Python antes de retornarlos, ya que DSDL serializa el resultado a JSON.

In [ ]:
# return model summary

def summary(model=None):
    """
    Proporcionar metadatos y resumen del modelo.
    
    Esta función es llamada automáticamente por DSDL para obtener información del modelo.
    DSDL serializa el resultado a JSON, por lo que todos los valores deben ser tipos nativos de Python.
    
    Args:
        model: Modelo Keras (opcional)
    
    Returns:
        dict: Metadatos del modelo
            - model_name: Nombre del modelo
            - app_name: Nombre de la aplicación
            - model_type: Tipo de modelo
            - use_case: Caso de uso
            - version: Versión del modelo
            - version_info: Versiones de librerías
            - model_summary: Resumen del modelo como string (si model está disponible)
            - model_architecture: Información de la arquitectura (si model está disponible)
            - layers: Información de cada capa (si model está disponible)
    """
    returns = {
        "model_name": MODEL_NAME,
        "app_name": APP_NAME,
        "model_type": MODEL_TYPE,
        "use_case": USE_CASE,
        "version": VERSION,
        "version_info": {
            "tensorflow": tf.__version__,
            "keras": keras.__version__,
            "numpy": np.__version__,
            "pandas": pd.__version__
        }
    }
    
    if model is not None:
        # Guardar resumen del modelo como string
        s = []
        model.summary(print_fn=lambda x: s.append(x + '\n'))
        returns["model_summary"] = ''.join(s)
        
        # Información de la arquitectura
        # ⚠️ CRÍTICO: Convertir valores NumPy a tipos nativos de Python para JSON serialization
        total_params = model.count_params()
        trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
        
        # Convertir a tipos nativos de Python
        if hasattr(total_params, 'item'):
            total_params = int(total_params.item())
        else:
            total_params = int(total_params)
        
        if hasattr(trainable_params, 'item'):
            trainable_params = int(trainable_params.item())
        else:
            trainable_params = int(trainable_params)
        
        returns["model_architecture"] = {
            "input_shape": str(model.input_shape) if hasattr(model, 'input_shape') else "N/A",
            "output_shape": str(model.output_shape) if hasattr(model, 'output_shape') else "N/A",
            "total_params": total_params,  # Ya convertido a int nativo
            "trainable_params": trainable_params  # Ya convertido a int nativo
        }
        
        # Información de capas
        returns["layers"] = []
        for i, layer in enumerate(model.layers):
            # Obtener output_shape de manera segura
            output_shape = "N/A"
            try:
                if hasattr(layer, 'output') and layer.output is not None:
                    try:
                        output_shape = str(layer.output.shape)
                    except:
                        pass
                
                if output_shape == "N/A":
                    if hasattr(layer, 'get_config'):
                        config = layer.get_config()
                        if 'output_shape' in config:
                            output_shape = str(config['output_shape'])
                
                if output_shape == "N/A":
                    if callable(getattr(layer, 'compute_output_shape', None)):
                        if i == 0 and hasattr(model, 'input_shape') and model.input_shape:
                            computed = layer.compute_output_shape(model.input_shape)
                            output_shape = str(computed)
                        elif hasattr(layer, 'input_shape') and layer.input_shape:
                            computed = layer.compute_output_shape(layer.input_shape)
                            output_shape = str(computed)
            except Exception:
                output_shape = "N/A"
            
            # Obtener parámetros de manera segura
            params = 0
            try:
                params_raw = layer.count_params()
                # ⚠️ CRÍTICO: Convertir a tipo nativo de Python para JSON serialization
                if hasattr(params_raw, 'item'):
                    params = int(params_raw.item())
                else:
                    params = int(params_raw)
            except Exception:
                params = 0
            
            returns["layers"].append({
                "index": i,  # Ya es int nativo
                "name": layer.name,
                "type": type(layer).__name__,
                "output_shape": output_shape,
                "params": params  # Ya convertido a int nativo
            })
    
    return returns

In [ ]:
# THIS CELL IS NOT EXPORTED - Test summary
# ⚠️ NOTA: Solo funciona si tienes modelo disponible

try:
    if 'model' not in globals():
        print("⚠️  No hay modelo disponible. Ejecuta primero init() y fit()")
    else:
        model_summary = summary(model)
        print("📊 Resumen del modelo:")
        print(json.dumps(model_summary, indent=2, default=str))
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## ✅ Checklist Antes de Guardar

Antes de guardar el notebook para publicación, verifica:

- [ ] **Configuración del modelo**: Actualiza `APP_NAME`, `MODEL_TYPE`, `USE_CASE`, `VERSION` en la celda de imports
- [ ] **Metadata de celdas**: Todas las funciones requeridas tienen el metadata correcto:
  - [ ] `init()` → metadata `"name": "mltkc_init"`
  - [ ] `fit()` → metadata `"name": "mltkc_stage_create_model_fit"`
  - [ ] `apply()` → metadata `"name": "mltkc_stage_create_model_apply"`
  - [ ] `summary()` → metadata `"name": "mltkc_summary"`
  - [ ] `save()` → metadata `"name": "mltkc_save"`
  - [ ] `load()` → metadata `"name": "mltkc_load"` (opcional)
- [ ] **Funciones requeridas**: Todas las funciones tienen las firmas correctas
- [ ] **Sin errores**: Ejecuta "Cell → Run All" para verificar que no hay errores
- [ ] **Nombre del notebook**: Sigue la convención `{app_name}_{model_type}_{use_case}_{version}.ipynb`

**Para más información**: Consulta la **Guía Completa Data Scientist E2E** para detalles sobre metadata y exportación.

---

## 📚 Recursos Adicionales

- **Guía Completa Data Scientist E2E**: `GUIA_COMPLETA_DATA_SCIENTIST_E2E.md`
- **Troubleshooting**: `TROUBLESHOOTING.md` - Solución de problemas comunes
- **Diagnóstico de Telemetría**: `DIAGNOSTICO_TELEMETRIA.md`
- **Documentación DSDL**: https://docs.splunk.com/Documentation/DSDL

---

## 🎯 Próximos Pasos

Una vez que el modelo está publicado:

1. **Monitorear métricas**: Revisar `index=ml_metrics` regularmente
2. **Ajustar thresholds**: Modificar percentil de anomalías según necesidad
3. **Crear dashboards**: Visualizar anomalías en tiempo real
4. **Configurar alertas**: Alertar cuando anomalías superen umbral
5. **Refinar modelo**: Iterar con más datos o arquitecturas diferentes

In [ ]:
# THIS CELL IS NOT EXPORTED - Celdas adicionales para desarrollo
# Puedes agregar aquí celdas adicionales para pruebas, visualizaciones, etc.
# Estas celdas NO se exportan al archivo .py

print("💡 Usa esta área para código de desarrollo que no debe exportarse")
print("   Ejemplo: análisis exploratorio, visualizaciones, pruebas adicionales")
